In [1]:
import psutil
import joblib
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
import torch.nn as nn
from sklearn.model_selection import KFold
from torch.utils.data import Dataset, DataLoader
import gc
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
#load data
data = pd.read_csv('../input/practice-log-demographics/Practice_Log_Demographics (1).csv', low_memory=False)
data.head()

,Unnamed: 0,chapter_label,sub_chapter_label,question_name,user_id.x,term,STDNT_SEX_CD,NonNativeEnglish,White,Asian,...,classGraded,classHonors,Pass_Fail,parentsGraduateEdu,minorityGroup,q,day's_available_flashcards,start_practice,end_practice,days_offset
0,1,Sort,intro-SortingwithSortandSorted,ee_sort_04,148,WN 2018,1,0,1,0,...,1,0,0,0,0,0,18,2018-03-04 20:00:19,2018-03-04 20:00:35,45
1,2,Functions,Returningavaluefromafunction,test_questionfunctions_2_3,148,WN 2018,1,0,1,0,...,1,0,0,0,0,4,16,2018-02-10 11:57:55,2018-02-10 11:58:35,23
2,3,RESTAPIs,functionswithrestapis,test_question_functions_rest_apis_1,148,WN 2018,1,0,1,0,...,1,0,0,0,0,1,2,2018-03-12 17:03:07,2018-03-12 17:04:28,53
3,4,SimplePythonData,FunctionCalls,exercise_functionCalls_1,148,WN 2018,1,0,1,0,...,1,0,0,0,0,0,10,2018-01-30 14:15:49,2018-01-30 14:15:58,12
4,5,DictionaryAccumulation,AccumulatingaMaximumValue,ps_04_07,148,WN 2018,1,0,1,0,...,1,0,0,0,0,0,23,2018-03-03 14:04:17,2018-03-03 14:04:25,44


In [3]:
data['term_value'] = [int(ele[3:]) for ele in data['term']]
data['age'] = data['term_value'] - data['birthYear']

In [4]:
#transform minor_count column
new_minors_count = []
for i in data['minorsCount']:
  if i == 0 or i == '0':
    new_minors_count.append(0)
  elif i == '1 Minor':
    new_minors_count.append(1)
  else:
    new_minors_count.append(2)

data['minorsCount'] = new_minors_count

In [5]:
FEATURES = ['chapter_label', 'sub_chapter_label','question_name','user_id','term',
            'STDNT_SEX_CD', 
            'NonNativeEnglish',
            'White',
            'Asian',
            'WhiteOrAsian',
            'Hispanic',
            'AfricanAmerican',
            'OtherEthnicities',
            'NonWhiteOrAsian',
            'STDNT_CTZN_STAT_CD', 'international', 
            'gradingType',
            'birthYear',
            'exclClassCumGPA',
            'Freshman',
            'Junior',
            'Sophomore',
            'Senior',
            'termCreditsGPA',
            'termCreditsNoGPA',
            'athlete_1',
            'honorsPro',
            'LSA', 'programBusiness', 'programEngineering', 
            'programInformation', 'programOther',
            'HSCalculusTaken', 
            'highSchoolGPA', 
            'majorsCount', 'minorsCount',
            'PREV_TERM_CUM_GPA',
            'classGraded', 'classHonors', 
            'Pass_Fail', 
            'parentsGraduateEdu',  'minorityGroup', 
            'q',
            'available_flashcards', 
            'start_practice', 
            'end_practice',
            'days_offset']

In [6]:
gc.collect()

44

In [7]:
#convert start_practice and end_practice to datatime column
data.start_practice = pd.to_datetime(data.start_practice, format='%Y-%m-%d %H:%M:%S')
data.end_practice = pd.to_datetime(data.end_practice, format='%Y-%m-%d %H:%M:%S')
#label encode term, chapter_label, question_name, sub_chapter_label columns
term_d = {'WN 2018': 0, 'FA 2018':1, 'WN 2019': 2, 'FA 2019':3, 'WN 2020': 4, 'FA 2020':5, 'WN 2021': 6 }
data['chapter_id'] = data['chapter_label'].replace(data['chapter_label'].unique(), list(range(data['chapter_label'].nunique()))) + 1
data['sub_chapter_id'] = data['sub_chapter_label'].replace(data['sub_chapter_label'].unique(), list(range(data['sub_chapter_label'].nunique()))) + 1
data['question_id'] = data['question_name'].replace(data['question_name'].unique(), list(range(data['question_name'].nunique()))) + 1
data['term'] = data['term'].replace(term_d) + 1

In [8]:
data['dif'] = data.end_practice - data.start_practice
data['dif'] = data['dif'] /np.timedelta64(1, 's')
data['q'] = np.where((data['q']==5) & (data['dif'] <= 60), 1, 0)

In [9]:
#drop chapter_label, sub_chapter_label, question_name
data.drop(columns = ['chapter_label', 'sub_chapter_label', 'question_name'], inplace=True)

In [10]:
#sort data based on timestamp
data = data.sort_values(['start_practice'], ascending=True).reset_index(drop=True)

In [11]:
#convert user_id column to the int type user_id column
data['user_id.x'] = data['term'].astype(str) + data['user_id.x'].astype(str)
data['user_id.x'] = data['user_id.x'].replace(data['user_id.x'].unique(), list(range(data['user_id.x'].nunique())))

In [12]:
data['prev_time_elapsed'] = None
data['time_lag'] = None
data['time_lag'] = data['time_lag'].astype(np.float)
data['prev_time_elapsed'] = data['prev_time_elapsed'].astype(np.float)

In [14]:
#sort data based on timestamp
data = data.sort_values(['start_practice'], ascending=True).reset_index(drop=True)

In [15]:
#now we can drop end practice
data.drop(columns=['end_practice'], inplace=True)
data = data.sort_values(['start_practice'], ascending=True).reset_index(drop=True)
#replace column q with anwered_correctly
data['answered_correctly'] = data['q']
data.drop(columns='q', inplace=True)
gc.collect()

70

In [16]:
gc.collect()

22

In [17]:
data.fillna(0, inplace=True)

In [18]:
n_skill = data['question_id'].nunique() + 1

In [37]:
#create dataset class
#to prepare it for train and valid sets
#here only original features are included
#that were present in SAKT: questions and answers
class PRACTICE_DATASET(Dataset):
    def __init__(self, group, n_skill=data['question_id'].nunique() + 1, min_samples=1, max_seq=2):
        super(PRACTICE_DATASET, self).__init__()
        self.max_seq = max_seq
        self.n_skill = n_skill        
        self.user_ids = group.keys()
        self.data = group
    
    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, index):
        user_id = self.user_ids[index]
        q_, qa_ = self.data[user_id]
        q_, qa_ = np.array(q_), np.array(qa_)
        seq_len = len(q_)

        q = np.zeros(self.max_seq, dtype=int)
        qa = np.zeros(self.max_seq, dtype=int)
        #if more the user interactions are more than self.maxlength (2)
        #only the last self.maxlength interactions will be included
        #if the number of user interactions would be less than self.maxlength
        #the padding would be utilized
        if seq_len >= self.max_seq:
            q[-self.max_seq:] = q_[-self.max_seq:]
            qa[-self.max_seq:] = qa_[-self.max_seq:]
        else:
            q[-seq_len:] = q_
            qa[-seq_len:] = qa_

        target_id = q[1:]
        label = qa[1:]

        x = np.zeros(self.max_seq-1, dtype=int)
        x = q[:-1].copy()
        x += (qa[:-1] == 1) * self.n_skill

        return x, target_id, label

In [29]:
NUMERIC_FEATURE =  ['age',
                    'exclClassCumGPA',
                    'termCreditsGPA',
                    'termCreditsNoGPA',
                    'highSchoolGPA', 
                    'majorsCount', 'minorsCount',
                    'PREV_TERM_CUM_GPA',
                    "day's_available_flashcards", 
                    'days_offset', 
                    'prev_time_elapsed',
                    'time_lag']
 
for f in NUMERIC_FEATURE:
  m = data[f].mean()
  std = data[f].std()
  data[f] = (data[f] - m)/std

In [30]:
#USE ONLY FEATURES PRESENT IN SAKT
group = data.groupby(['user_id.x']).apply(lambda r: (
                r['question_id'].values,
                r['answered_correctly'].values
                ))

In [38]:
#feed forward network
class FFN(nn.Module):
    def __init__(self, state_size=200):
        super(FFN, self).__init__()
        self.state_size = state_size

        self.lr1 = nn.Linear(state_size, state_size)
        self.relu = nn.ReLU()
        self.lr2 = nn.Linear(state_size, state_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        return self.dropout(x)
    
#define mask that would be used in multi head attention layer
def future_mask(seq_length):
    future_mask = np.triu(np.ones((seq_length, seq_length)), k=1).astype('bool')
    return torch.from_numpy(future_mask)

#define the SAKT model
class SAKTModel(nn.Module):
    def __init__(self, n_skill, max_seq=2, embed_dim=128, dropout_rate=0.2):
        super(SAKTModel, self).__init__()
        self.n_skill = n_skill
        self.embed_dim = embed_dim

        #embeddings
        self.embedding = nn.Embedding(2*n_skill+1, embed_dim) 
        self.pos_embedding = nn.Embedding(max_seq-1, embed_dim)
        self.e_embedding = nn.Embedding(n_skill+1, embed_dim)

        self.multi_att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8, dropout=dropout_rate)

        self.dropout = nn.Dropout(dropout_rate)
        self.layer_normal = nn.LayerNorm(embed_dim) 

        self.ffn = FFN(embed_dim)
        self.pred = nn.Linear(embed_dim, 1)
    
    def forward(self, x, question_ids):
        device = x.device        
        x = self.embedding(x)
        pos_id = torch.arange(x.size(1)).unsqueeze(0).to(device)

        pos_x = self.pos_embedding(pos_id)
        x = x + pos_x

        e = self.e_embedding(question_ids)

        x = x.permute(1, 0, 2) 
        e = e.permute(1, 0, 2)
        att_mask = future_mask(x.size(0)).to(device)
        att_output, att_weight = self.multi_att(e, x, x, attn_mask=att_mask)
        att_output = self.layer_normal(att_output + e)
        att_output = att_output.permute(1, 0, 2) 

        x = self.ffn(att_output)
        x = self.layer_normal(x + att_output)
        x = self.pred(x)

        return x.squeeze(-1)

In [39]:
# remove students who don't have make any interactions with the tool
toRemove = []
for index in group.index:
  if len(group[index][0]) <= 10:
    toRemove.append(index)
group = group.drop(index=toRemove)

In [40]:
#in case the validation loss would 
#not improve for $patienc$ consecutive 
#epochs the training would be terminated
patience = 5

In [41]:
#user KFold
X = np.array(group.keys())
kfold = KFold(n_splits=5, shuffle=True)
train_losses = list()
train_aucs = list()
train_accs = list()
val_losses = list()
val_aucs = list()
val_accs = list()
test_losses = list()
test_aucs = list()
test_accs = list()
for train, test in kfold.split(X):
    users_train, users_test =  X[train], X[test]
    n = len(users_test)//2
    users_test, users_val = users_test[:n], users_test[n: ]
    train = PRACTICE_DATASET(group[users_train])
    valid = PRACTICE_DATASET(group[users_val])
    test = PRACTICE_DATASET(group[users_test])
    train_dataloader = DataLoader(train, batch_size=64, shuffle=True, num_workers=8)
    val_dataloader = DataLoader(valid, batch_size=64, shuffle=True, num_workers=8)
    test_dataloader = DataLoader(test, batch_size=64, shuffle=True, num_workers=8)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    saint = SAKTModel(n_skill)
    epochs = 100
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(saint.parameters(), betas=(0.9, 0.999), lr = 0.0005, eps=1e-8)
    saint.to(device)
    criterion.to(device)
    
    def train_epoch(model=saint, train_iterator=train_dataloader, optim=optimizer, criterion=criterion, device=device):
        model.train()

        train_loss = []
        num_corrects = 0
        num_total = 0
        labels = []
        outs = []
        tbar = tqdm(train_iterator)
        for item in tbar:
            x = item[0].to(device).long()
            target_id = item[1].to(device).long()
            label = item[2].to(device).float()            
            target_mask = (target_id!=0)
            optim.zero_grad()
            output = model(x, target_id)
            output = torch.reshape(output, label.shape)

            output = torch.masked_select(output, target_mask)
            label = torch.masked_select(label, target_mask)

            loss = criterion(output, label)
            loss.backward()
            optim.step()
            train_loss.append(loss.item())
            pred = (torch.sigmoid(output) >= 0.5).long()

            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

            tbar.set_description('loss - {:.4f}'.format(loss))
        acc = num_corrects / num_total
        auc = roc_auc_score(labels, outs)
        loss = np.mean(train_loss)

        return loss, acc, auc
   

    def val_epoch(model=saint, val_iterator=test_dataloader, 
              criterion=criterion, device=device):
        model.eval()

        train_loss = []
        num_corrects = 0
        num_total = 0
        labels = []
        outs = []
        tbar = tqdm(val_iterator)
        for item in tbar:
            x = item[0].to(device).long()
            target_id = item[1].to(device).long()
            label = item[2].to(device).float()                
            target_mask = (target_id!=0)
            with torch.no_grad():
                output = model(x, target_id)

            output = torch.reshape(output, label.shape)
            output = torch.masked_select(output, target_mask)
            label = torch.masked_select(label, target_mask)

            loss = criterion(output, label)
            train_loss.append(loss.item())

            pred = (torch.sigmoid(output) >= 0.5).long()
            num_corrects += (pred == label).sum().item()
            num_total += len(label)

            labels.extend(label.view(-1).data.cpu().numpy())
            outs.extend(output.view(-1).data.cpu().numpy())

            tbar.set_description('valid loss - {:.4f}'.format(loss))

        acc = num_corrects / num_total
        auc = roc_auc_score(labels, outs)
        loss = np.average(train_loss)

        return loss, acc, auc
    
    MIN_VAL = 1000000000
    count = 0
    print('----------------------------------------------------------------------------')
    for epoch in range(epochs):
        train_loss, train_acc, train_auc = train_epoch(model=saint, device=device)
        print("epoch - {} train_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, train_loss, train_acc, train_auc))
        val_loss, val_acc, val_auc = val_epoch(model=saint, val_iterator= val_dataloader, device=device)
        print("epoch - {} val_loss - {:.2f} val acc - {:.3f} val auc - {:.3f}".format(epoch, val_loss, val_acc, val_auc))
        if val_loss < MIN_VAL:
            count = 0
            MIN_VAL = val_loss
        else:
            count += 1
        if count == patience:
            print('Val Loss does not improve for {} consecutive epochs'.format(patience))
            break
    test_loss, test_acc, test_auc = val_epoch(model=saint, device=device)
    print("epoch - {} test_loss - {:.2f} acc - {:.3f} auc - {:.3f}".format(epoch, test_loss, test_acc, test_auc))
    train_loss, train_acc, train_auc = val_epoch(model=saint, val_iterator=train_dataloader, device=device)

    test_losses.append(test_loss)
    test_aucs.append(test_auc)
    test_accs.append(test_acc)
    train_aucs.append(train_auc)
    train_losses.append(train_loss)
    train_accs.append(train_acc)

  0%|          | 0/14 [00:00<?, ?it/s]

----------------------------------------------------------------------------


  0%|          | 0/2 [00:00<?, ?it/s]

epoch - 0 train_loss - 0.65 acc - 0.683 auc - 0.471


  0%|          | 0/14 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

epoch - 11 val_loss - 0.63 val acc - 0.703 val auc - 0.685
Val Loss does not improve for 5 consecutive epochs


  0%|          | 0/14 [00:00<?, ?it/s]

epoch - 11 test_loss - 0.65 acc - 0.727 auc - 0.593


valid loss - 0.2366: 100%|██████████| 14/14 [00:00<00:00, 33.47it/s]


In [42]:
#display test loss/acc/auc
print("test avg loss: ", np.mean(test_losses), np.std(test_losses) )
print("test avg acc: ", np.mean(test_accs), np.std(test_accs))
print("test avg auc: ", np.mean(test_aucs), np.std(test_aucs))

test avg loss:  0.7390848159790039 0.067940706448927
test avg acc:  0.670941850941851 0.031123255288653062
test avg auc:  0.567761879243022 0.042992323435332915


In [43]:
#display train loss/acc/auc
print("train avg loss: ", np.mean(train_losses), np.std(train_losses) )
print("train avg acc: ", np.mean(train_accs), np.std(train_accs))
print("train avg auc: ", np.mean(train_aucs), np.std(train_aucs))

train avg loss:  0.2875658690929413 0.03961655778124074
train avg acc:  0.887630458516673 0.02427602624521948
train avg auc:  0.9483967183942361 0.018393370201960205
